In [1]:
import pandas as pd

df = pd.read_csv('cleaned_altmetrics.csv', delimiter=';')
df = df.fillna(0)
df

,id,altmetric_score,title,journal_name,print_issn,online_issn,subjects,affiliation,publication_date,doi,...,reddit,pinterest,f1000,qa,videos,syllabi,mendeley,dimensions_citations,details_page,badge_url
0,4,995,Is Noise Always Bad? Exploring the Effects of ...,Journal of Consumer Research,1537-5277,0093-5301,17 Psychology and Cognitive Sciences; 1701 Psy...,0,01-12-2012,10.1086/665048,...,7,0,0,0,1,0,419,74,https://www.altmetric.com/details/1575589,https://api.altmetric.com/v1/donut/1575589_240...
1,8,830,The Role of Application Assistance and Informa...,Quarterly Journal of Economics,1531-4650,0033-5533,0,0,18-04-2012,10.1093/qje/qjs017,...,0,0,0,0,0,0,243,220,https://www.altmetric.com/details/704830,https://api.altmetric.com/v1/donut/704830_240.png
2,10,760,Are You Happy While You Work?,Economic Journal,0013-0133,1468-0297,08 Information and Computing Sciences; 0801 Ar...,University of Sussex,12-01-2016,10.1111/ecoj.12269,...,0,0,0,0,0,0,77,22,https://www.altmetric.com/details/3910209,https://api.altmetric.com/v1/donut/3910209_240...
3,12,715,Whitened R?sum?s,Administrative Science Quarterly,0001-8392,1930-3815,17 Psychology and Cognitive Sciences; 1701 Psy...,Stanford University; University of Toronto,08-07-2016,10.1177/0001839216639577,...,2,0,0,0,0,0,211,29,https://www.altmetric.com/details/6204393,https://api.altmetric.com/v1/donut/6204393_240...
4,14,699,The Fundamental Law of Road Congestion: Eviden...,American Economic Review,0002-8282,0,0,0,01-10-2011,10.1257/aer.101.6.2616,...,0,0,0,0,0,0,324,211,https://www.altmetric.com/details/1496402,https://api.altmetric.com/v1/donut/1496402_240...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13922,98056,0,Humorous advertising that travels: A review an...,Journal of Business Research,0148-2963,0,"20 Language, Communication and Culture; 2002 C...",Charles Sturt University; UNSW Sydney,01-03-2015,10.1016/j.jbusres.2014.09.005,...,0,0,0,0,0,0,72,14,https://www.altmetric.com/details/2764466,https://api.altmetric.com/v1/donut/2764466_240...
13923,98057,0,The moderating effect of individual level coll...,Journal of Business Research,0148-2963,0,"15 Commerce, Management, Tourism and Services;...",King's College London; Monash University; Univ...,01-11-2014,10.1016/j.jbusres.2014.02.011,...,0,0,0,0,0,0,98,10,https://www.altmetric.com/details/2714737,https://api.altmetric.com/v1/donut/2714737_240...
13924,98107,0,Symbolic consumption of tourism destination br...,Journal of Business Research,0148-2963,0,"15 Commerce, Management, Tourism and Services;...",Oxford Brookes University; University of South...,01-06-2013,10.1016/j.jbusres.2011.09.008,...,0,0,0,0,0,0,193,78,https://www.altmetric.com/details/1778262,https://api.altmetric.com/v1/donut/1778262_240...
13925,98122,0,Consumer roles in brand culture and value co-c...,Journal of Business Research,0148-2963,0,"15 Commerce, Management, Tourism and Services;...",Dublin City University,01-09-2013,10.1016/j.jbusres.2012.09.014,...,0,0,0,0,0,0,357,56,https://www.altmetric.com/details/1497264,https://api.altmetric.com/v1/donut/1497264_240...


In [2]:
# Hightest Score Per Journal
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool

output_notebook()

journals = list(set(df['journal_name']))[1:]

journals_max = zip(journals, map(lambda x: max([value['altmetric_score'] for item, value in df.iterrows() if value['journal_name'] == x]), journals))

d = dict(journals_max)
data = pd.DataFrame(d.items(), columns=['Journal', 'Score'])

hover = HoverTool(
            tooltips=[
                ("Journal", "@Journal"),
                ("Alt. Score", "@Score")
            ]
        )

p = figure(x_range=data.Journal, plot_height=850, plot_width=850, title="Highest Score per Journal", tools=[hover])
p.vbar(x='Journal', top='Score', width=0.9, source=data)
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = 1

show(p)

Loading BokehJS ...

In [3]:
from math import pi

from bokeh.palettes import Category20c
from bokeh.transform import cumsum

subset = df[df.journal_name == "Journal of Consumer Research"]
tracking = df.columns[10:28]
x = {t: sum(subset[t]) for t in tracking if t != 'mendeley'}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'platform'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=850, plot_width=950, title="Journal of Consumer Research Engagement (minus Mendeley)", toolbar_location=None,
           tools="hover", tooltips="@platform: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='platform', source=data)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color=None

show(p)

In [10]:
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter, LogTickFormatter
from bokeh.transform import transform
from bokeh.sampledata.unemployment1948 import data

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# group
d = df
d = d.groupby('journal_name')[[t for t in tracking]].mean()
d = d.drop(0)
d = pd.DataFrame(scaler.fit_transform(d), columns=d.columns, index=d.index)

In [13]:
data = d
data.columns.name = "Source"

d = pd.DataFrame(data.stack(), columns=['rate']).reset_index()
source = ColumnDataSource(d)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette="Viridis256", low=d.rate.min(), high=d.rate.max())


p = figure(plot_width=900, plot_height=800, title="Engagement Across Journals (MinMaxScaled)",
           x_range=list(data.index), y_range=list(reversed(data.columns)),
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="journal_name", y="Source", width=1, height=1, source=source,
       line_color=None, fill_color=transform('rate', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(tracking)),
                     formatter=PrintfTickFormatter(format="%e"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

show(p)

In [42]:
from statistics import mean

journals_avg = zip(journals, map(lambda x: round(mean([value['altmetric_score'] for item, value in df.iterrows() if value['journal_name'] == x]), 2), journals))

d = dict(journals_avg)
data = pd.DataFrame(d.items(), columns=['Journal', 'Avg. Score'])
d



{'Administrative Science Quarterly': 28.93,
 'Journal of the American Statistical Association': 4.42,
 'Journal of Financial Economics': 4.06,
 'Journal of Health Economics': 16.4,
 'Quarterly Journal of Economics': 47.81,
 'Academy of Management Review': 5.94,
 'Journal of Business Research': 3.04,
 'Journal of the European Economic Association': 9.51,
 'Journal of Marketing': 12.46,
 'Journal of Labor Economics': 15.23,
 'The Review of Economic Studies': 2,
 'Journal of Finance': 7.18,
 'Journal of Econometrics': 2.16,
 'Journal of Political Economy': 16.01,
 'Econometrica': 4.26,
 'Journal of Accounting & Economics': 2.44,
 'The Quarterly Journal of Economics': 1,
 'Journal of Monetary Economics': 3.52,
 'Information Systems Research': 5.55,
 'European Economic Review': 5.86,
 'The Academy of Management Annals': 5.71,
 'Review of Economic Studies': 11.48,
 'Games & Economic Behavior': 1.72,
 'Economic Journal': 17.06,
 'Journal of Consumer Research': 35,
 'Journal of Economic Theory